##Create table for OLTP "trips" data

In [0]:
table_name = "stations"

Remove loaded delta files from silver storage to satisfy overwrite requirements

In [0]:
%fs rm -r lakehouse/silver/oltp_stations

res0: Boolean = false

In [0]:
raw_stations_df = spark.read.format('delta').load(F"/lakehouse/bronze/{table_name}")

In [0]:
raw_stations_df.count()

838

In [0]:
raw_stations_df.createOrReplaceTempView(F'oltp_{table_name}')

In [0]:
# to satisfy overwrite requirements
spark.sql(F'''
           DROP TABLE IF EXISTS silver_{table_name}  
          ''')


DataFrame[]

In [0]:
#Creating empty table
spark.sql(F''' CREATE TABLE silver_{table_name}
                (
                    station_id STRING,
                    name STRING,
                    latitude FLOAT, 
                    longitude FLOAT 
                )
                USING delta
                LOCATION "/lakehouse/silver/oltp_{table_name}" 
        ''')


DataFrame[]

In [0]:
spark.sql(F'''
          INSERT INTO silver_{table_name} SELECT * FROM oltp_{table_name}
          ''')

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
display(spark.sql(F'''
          SELECT count(*) FROM silver_{table_name}
          '''))

count(1)
838


In [0]:
# Checking for Null values
# "account_end_date" is null when the account is active
# Not under the WHERE statement, because null is 
# significant is its case
display(spark.sql('''
          SELECT 
            station_id, name, latitude, longitude
          FROM 
            silver_stations
          WHERE
            name IS NULL

        '''))

station_id,name,latitude,longitude


In [0]:
#checking for duplicates
display(spark.sql(''' 
                  SELECT 
                    station_id 
                  FROM 
                    silver_stations 
                  GROUP BY station_id
                  HAVING count(station_id) > 1  
                  '''
                ))

station_id
